This notebook can be used to access Landsat collections to download .tif scenes and convert those scenes into .csv files. The bias corrections applied to the data are calculated in the xxx notebook. These scenes are those found in the Non-Infilled folder, and the scenes are used in the infilling_dineof notebook.

In [21]:
# imports, connects to google drive and earth engine
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import numpy as np
import ee
import csv
!pip install rasterio
import rasterio
ee.Authenticate()
ee.Initialize()

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=cMQrZuh2UmkJj7HLmHdLDmCJpK3x900P7qKJDFS4q9M&tc=BZfdZJvBovxPgwRMXGB5Ix4hOeDjLDZUdXjyNX20Peo&cc=7Jtcc8_eFGEZu_KOID0jNqXn-ZvLApahOsbWacrTtBE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdexTELx2glgx57UW_NJdQncwp8KVxEftvriJglu3HPBhWDrG72eBTE

Successfully saved authorization token.


In [ ]:
#@title Geometry of Narragansett Bay (run this code)

NG0 = ee.Geometry.Polygon(
        [[-71.23430897643202, 41.67803448724837],
          [-71.2387115097295, 41.66434415508815],
          [-71.24506298067676, 41.66844770361683],
          [-71.25931087496387, 41.64279623448692],
          [-71.255191001917, 41.63663836177501],
          [-71.23407665255176, 41.63933250348853],
          [-71.2196570968877, 41.65626452960499],
          [-71.21656719210254, 41.639075923415746],
          [-71.21141735079395, 41.63933250348853],
          [-71.20778129628407, 41.65320086118295],
          [-71.19507835438954, 41.68166876899119],
          [-71.17947308941616, 41.69648952647567],
          [-71.17621152325405, 41.69956567534286],
          [-71.17037503643765, 41.7022571849021],
          [-71.16625516339077, 41.70558937398956],
          [-71.17655484600796, 41.7109717761861],
          [-71.17844312115444, 41.70879323910567],
          [-71.18696944419253, 41.71118570860874],
          [-71.18937270346987, 41.71118570860874],
          [-71.18748442832339, 41.71080126753273],
          [-71.1878277510773, 41.708110115624876],
          [-71.19126097861636, 41.70721303995403],
          [-71.19675414267886, 41.7061877952885],
          [-71.1965824813019, 41.70887902766563],
          [-71.1965824813019, 41.71310787950142],
          [-71.19349257651675, 41.71554254678427],
          [-71.19829909507143, 41.717080183867495],
          [-71.2043072432648, 41.707084885264926],
          [-71.21769683066714, 41.71157014738558],
          [-71.21477858725893, 41.71579882218616],
          [-71.2101437300812, 41.719770960243686],
          [-71.22439162436831, 41.72502405608457],
          [-71.2240483016144, 41.70836642066032],
          [-71.22971312705386, 41.70170215764726],
          [-71.24413268271792, 41.696318979413846]])
NG1 = ee.Geometry.Polygon(
        [[-71.21916723431646, 41.623793220398916],
          [-71.24010992230474, 41.621226755119096],
          [-71.24422979535161, 41.5860558955723],
          [-71.23427343548833, 41.57655390853521],
          [-71.23873663128911, 41.556004292188895],
          [-71.23427343548833, 41.55292128602609],
          [-71.23873663128911, 41.536733091185425],
          [-71.22757917871408, 41.53005508691169],
          [-71.24680525293283, 41.493805939196726],
          [-71.24577528467111, 41.48711948268164],
          [-71.2419128317205, 41.48589796269792],
          [-71.23805041709277, 41.486476627672694],
          [-71.23633390893869, 41.48416179140002],
          [-71.24045444215017, 41.47464467756757],
          [-71.24388745573923, 41.47380876598267],
          [-71.2480068825715, 41.470657572674504],
          [-71.24766377236483, 41.47618793953888],
          [-71.24594694604806, 41.48107536090158],
          [-71.2554734395774, 41.48699134899175],
          [-71.26688963403635, 41.48891975040266],
          [-71.2686496271165, 41.48467576210508],
          [-71.27367066674788, 41.47940322153438],
          [-71.27650267114572, 41.479146267189186],
          [-71.2835407876008, 41.48891975040269],
          [-71.28972072084748, 41.48814833674577],
          [-71.2948704384797, 41.48609073581],
          [-71.2970173622729, 41.48518902099484],
          [-71.29654574629426, 41.47805082049959],
          [-71.29768327866556, 41.4749962009723],
          [-71.29722206601771, 41.47205408365912],
          [-71.29573095434897, 41.470398174058744],
          [-71.29749077013402, 41.46756848928703],
          [-71.30128880044577, 41.46692547184332],
          [-71.3037026916472, 41.463388047586896],
          [-71.30233997938153, 41.460365177628596],
          [-71.3110066079133, 41.45058896659164],
          [-71.1959934853547, 41.453419530780195],
          [-71.19324690332346, 41.484033193064256],
          [-71.20732313623361, 41.49843462019324],
          [-71.2014866494172, 41.512832845653556],
          [-71.20903975000314, 41.527741918012794],
          [-71.20510983017888, 41.54097445846128],
          [-71.21403622178045, 41.55562013441321],
          [-71.20510983017888, 41.59388877253466],
          [-71.21334957627263, 41.61545304359883],
          [-71.21403622178045, 41.62546257669854],
          [-71.21197628525701, 41.63675351803523],
          [-71.21781277207342, 41.63675351803523]])
NG2 = ee.Geometry.Polygon(
        [[-71.26043798204432, 41.641597207901086],
          [-71.26764775987635, 41.645189095419994],
          [-71.27142431016932, 41.65314184809158],
          [-71.26902105089198, 41.65904164307961],
          [-71.28000737901698, 41.67237831781557],
          [-71.28035070177089, 41.68545583458813],
          [-71.28893377061854, 41.679814878769044],
          [-71.28996373888026, 41.65981114171309],
          [-71.29477025743495, 41.66211958244548],
          [-71.2978601622201, 41.6505765512707],
          [-71.30369664903651, 41.65365489519655],
          [-71.30529344227884, 41.67971936121889],
          [-71.29877030995462, 41.68177067814172],
          [-71.29156053212259, 41.69587171182405],
          [-71.2919038548765, 41.71176371651426],
          [-71.28572404530618, 41.714582855469594],
          [-71.28469407704446, 41.72765179322909],
          [-71.28744065907571, 41.730726451130586],
          [-71.29533708241556, 41.720989529620894],
          [-71.29362046864603, 41.71535169008936],
          [-71.29877030995462, 41.71919572523648],
          [-71.30220353749368, 41.705868753703975],
          [-71.30529344227884, 41.71996450467537],
          [-71.33825242665384, 41.72534570323228],
          [-71.3582206842159, 41.74562666036468],
          [-71.3692070123409, 41.76406723767318],
          [-71.37195359437214, 41.783014532072656],
          [-71.37108541179893, 41.78633971828446],
          [-71.39305806804893, 41.81065448099082],
          [-71.38556121021891, 41.8170096529418],
          [-71.3797247234025, 41.826220343852654],
          [-71.36908171803141, 41.837731844328694],
          [-71.37217162281657, 41.845149269877645],
          [-71.36908171803141, 41.85256583560228],
          [-71.38109801441813, 41.86995303154864],
          [-71.37800810963297, 41.832104258301364],
          [-71.38899443775797, 41.81828899479544],
          [-71.40341399342203, 41.81419501090403],
          [-71.39483092457438, 41.79781645839315],
          [-71.39277098805094, 41.80011994533946],
          [-71.37732146412516, 41.786041787606806],
          [-71.39395956174398, 41.78096111480256],
          [-71.37630275258908, 41.74421190392843],
          [-71.38042262563596, 41.73422082565026],
          [-71.3883190489758, 41.734733226451105],
          [-71.37355617055783, 41.720640714979034],
          [-71.3550167418469, 41.71679676626381],
          [-71.3663463927258, 41.71218372431149],
          [-71.36565974721799, 41.693471882391215],
          [-71.36325648794065, 41.69013906538831],
          [-71.3718395567883, 41.68475491930977],
          [-71.37814736619767, 41.66606548333479],
          [-71.38879037156876, 41.671964094393275],
          [-71.39188027635392, 41.68222126108208],
          [-71.44887185350235, 41.68863116011628],
          [-71.4461252714711, 41.675297852875495],
          [-71.44097543016251, 41.675297852875495],
          [-71.44097543016251, 41.67324632962789],
          [-71.4457819487172, 41.67222054348357],
          [-71.44406533494767, 41.658883836062905],
          [-71.44097543016251, 41.667091367559586],
          [-71.42552590623673, 41.67068183361664],
          [-71.40835976854142, 41.66221827190633],
          [-71.40973305955704, 41.6499056506324],
          [-71.40269621933261, 41.62525999113271],
          [-71.41642912948886, 41.60318555369872],
          [-71.40475615585605, 41.60342027932765],
          [-71.40441283310214, 41.58904221824255],
          [-71.37282713974277, 41.57389043902059],
          [-71.33128508652011, 41.5792838526816],
          [-71.34055480087558, 41.59623164915919],
          [-71.32750853622714, 41.62215829969842],
          [-71.34501799667636, 41.63191018344109],
          [-71.35703429306308, 41.637042161357876],
          [-71.3525710972623, 41.64935724119648],
          [-71.34948119247714, 41.66269592155889],
          [-71.34563036797367, 41.668012850890925],
          [-71.34116717217289, 41.66339630922525],
          [-71.33773394463383, 41.659292438745766],
          [-71.3367039763721, 41.65364918979668],
          [-71.33327074883304, 41.64672270847792],
          [-71.3308674895557, 41.64056521098952],
          [-71.33636044446534, 41.64210477345174],
          [-71.3404804337115, 41.64492697192849],
          [-71.34254046318851, 41.64261777553225],
          [-71.34271200017413, 41.640629290488604],
          [-71.33979370619642, 41.63915393687433],
          [-71.33704729912601, 41.63440712508837],
          [-71.31576128838383, 41.63081463658636],
          [-71.3011012345076, 41.61497404378497],
          [-71.31174423987869, 41.592125788257775],
          [-71.32753708655838, 41.57825885829603],
          [-71.29698136146072, 41.57004001301841],
          [-71.28633835608963, 41.575690581486505],
          [-71.28771164710525, 41.59186902032359],
          [-71.2809532684875, 41.59328837105911],
          [-71.27134023137812, 41.61382612656979],
          [-71.27820668645624, 41.619472863306335],
          [-71.26413045354609, 41.62640227360353],
          [-71.25726399846796, 41.63589692271017]])
NG3 = ee.Geometry.Polygon(
        [[-71.29675158769878, 41.56988128709373],
          [-71.33039721758159, 41.57912743721889],
          [-71.3695360115269, 41.573990850552384],
          [-71.36266955644878, 41.57116555375916],
          [-71.35751971514019, 41.54778790441438],
          [-71.36644610674175, 41.541620961293475],
          [-71.36005066737624, 41.524922177323056],
          [-71.36657379970046, 41.51309750716886],
          [-71.35593079432937, 41.5110408222542],
          [-71.35744513260191, 41.50769856990229],
          [-71.32448614822691, 41.49998502133089],
          [-71.33066595779722, 41.50589882409012],
          [-71.3296359895355, 41.51618241202012],
          [-71.32620276199644, 41.523894030902866],
          [-71.31761969314879, 41.5190101123084],
          [-71.31143988357847, 41.52235178064106],
          [-71.31658972488707, 41.53700167400588],
          [-71.30972326980894, 41.54059938594597],
          [-71.31212652908629, 41.54753726520721]])
NG4 = ee.Geometry.Polygon(
        [[-71.40395454224509, 41.58893436534363],
          [-71.43142027249172, 41.58225775897979],
          [-71.44137659972986, 41.575580462222746],
          [-71.43691341857212, 41.570957313870885],
          [-71.44206324301884, 41.565563222606826],
          [-71.43416684556725, 41.56042557410598],
          [-71.42730041299404, 41.55528751712591],
          [-71.42798705626147, 41.549635182667956],
          [-71.42318055345467, 41.54398235396264],
          [-71.42043398042166, 41.537558085063296],
          [-71.41450909485361, 41.522643574596636],
          [-71.41519573811613, 41.50953306531142],
          [-71.41978790060877, 41.50313712472273],
          [-71.4207104459297, 41.498010649508764],
          [-71.41905810591241, 41.49224111945568],
          [-71.4197233019943, 41.486127939418246],
          [-71.41764202998182, 41.48181506281701],
          [-71.41710564613979, 41.47595898277189],
          [-71.42068888424141, 41.471131794434015],
          [-71.42927192505572, 41.47113179443995],
          [-71.4298861903909, 41.45988008238931],
          [-71.42473636590063, 41.45576328079208],
          [-71.43194612021875, 41.445212783977695],
          [-71.43125947695158, 41.44521278397695],
          [-71.43125947695209, 41.444955433351],
          [-71.42782626061685, 41.444440729033694],
          [-71.42164647120985, 41.44649952178832],
          [-71.40207713809896, 41.44727155223416],
          [-71.40207713809896, 41.44727155223416],
          [-71.40070385159014, 41.46348206941929],
          [-71.39555402710256, 41.466054794810276],
          [-71.40139049489694, 41.49126210134013],
          [-71.3862843431078, 41.494090880894205],
          [-71.37838794563248, 41.506947412773485],
          [-71.39561447598935, 41.51574816346719],
          [-71.39218125971325, 41.528343375655766],
          [-71.38758906612135, 41.5396995556953],
          [-71.38243924177505, 41.56744690145359],
          [-71.37419952271426, 41.57386820041247]])
NG5 = ee.Geometry.Polygon(
        [[-71.40096554744181, 41.447468987169984],
          [-71.35736355769572, 41.448627015864815],
          [-71.36423001277385, 41.45994887397815],
          [-71.36114010798869, 41.4661236000062],
          [-71.33813748347697, 41.481300635368406],
          [-71.33161435115275, 41.47898572406981],
          [-71.33676419246135, 41.46946799777046],
          [-71.31719479548869, 41.47847128810737],
          [-71.32406125056681, 41.497245555123975],
          [-71.3559902666801, 41.505216543340616],
          [-71.36388669001994, 41.499302678268535],
          [-71.35805020320353, 41.48181504887193],
          [-71.35805020320353, 41.476413503423544],
          [-71.37143979060588, 41.47564181732317],
          [-71.37624630916056, 41.47435565340574],
          [-71.38139615046916, 41.48747332789982],
          [-71.38654599177775, 41.48618739878098],
          [-71.38036618220744, 41.46946799777046],
          [-71.38517270076213, 41.463808146648724],
          [-71.38963589656291, 41.46483724723775],
          [-71.39032254207072, 41.456861290476624]])

NGBayGeom = NG0.union(NG1).union(NG2).union(NG3).union(NG4).union(NG5)

# Accessing .tif Data

In [ ]:
# functions for getting and processing Landsat data

# start and end date of data, as well as location
date_start = '1984-01-01'
date_end = '2023-01-01'
path = 12
row = 31

# function that applies land mask
def land_mask(image):
  hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
  datamask = hansenImage.select('datamask')
  mask = ee.Image(datamask.eq(2))
  return image.mask(mask)
  
# function that gets qa bits (metadata used for clouds)
def get_qa_bits(image, start, end, mascara):
  pattern = 0
  for i in range(start,end+1):
    pattern += 2**i
  return image.select([0],[mascara]).bitwiseAnd(pattern).rightShift(start)

# function that applies cloud mask on Landsat 5 and 7
def L57_cloud_mask(image):
  QA = image.select('QA_PIXEL') 
  cloud = get_qa_bits(QA,3,3,'cloud')           
  shadow = get_qa_bits(QA,4,4,'cloud_shadow')
  image = image.updateMask(shadow.eq(0)).updateMask(cloud.eq(0))
  return image.addBands(cloud.rename("cloud"))

# function that applies cloud mask on Landsat 8
def L8_cloud_mask(image):
  QA = image.select('QA_PIXEL') 
  cirrus = get_qa_bits(QA,2,2,'cirrus_detected') 
  cloud = get_qa_bits(QA,3,3,'cloud')           
  shadow = get_qa_bits(QA,4,4,'cloud_shadow')
  image = image.updateMask(shadow.eq(0)).updateMask(cloud.eq(0).updateMask(cirrus.eq(0)))
  return image.addBands(cloud.rename("cloud"))

# function that applies scaling factors to Landsat 5 and 7
def L57_scale_factors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBand, None, True)

# function that applies scaling factors to Landsat 8
def L8_scale_factors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# function that calculates the percent cloud cover over the bay
def percent_cloud(image):
  MeanCloudNG = image.select(["cloud"]).reduceRegion(geometry = NGBayGeom, 
                                                     reducer=ee.Reducer.mean(), 
                                                     crs='EPSG:4326', 
                                                     scale=500).getInfo()
  return float(round(MeanCloudNG["cloud"]*100, 2))

# function that gets a list of cloud cover percentages for a collection
def get_landsat_clouds(collection):
  dates = ee.List(collection.aggregate_array("DATE_ACQUIRED")).getInfo()
  clouds = np.ones(len(dates))*np.nan
  for i in range(len(dates)):
    filtered = collection.filterDate(dates[i], date_end)
    clouds[i] = percent_cloud(filtered.first())
  return(clouds)

# function that gets a list of dates for a collection
def get_landsat_dates(collection):
  dates = ee.List(collection.aggregate_array("DATE_ACQUIRED")).getInfo()
  return dates

In [ ]:
# applies the land and cloud masks, filters by date, row, and path,
# and applies the scaling factors to all three Landsat collections
L5_masked = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')\
  .filter(ee.Filter.eq('WRS_PATH', path))\
  .filter(ee.Filter.eq('WRS_ROW', row))\
  .filterDate(date_start, date_end)\
  .map(L57_scale_factors)\
  .map(land_mask)\
  .map(L57_cloud_mask)
L7_masked = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
  .filter(ee.Filter.eq('WRS_PATH', path))\
  .filter(ee.Filter.eq('WRS_ROW', row))\
  .filterDate(date_start, date_end)\
  .map(L57_scale_factors)\
  .map(land_mask)\
  .map(L57_cloud_mask)
L8_masked = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
  .filter(ee.Filter.eq('WRS_PATH', path))\
  .filter(ee.Filter.eq('WRS_ROW', row))\
  .filterDate(date_start, date_end)\
  .map(L8_scale_factors)\
  .map(land_mask)\
  .map(L8_cloud_mask)

In [ ]:
# gets and saves the date and cloud cover percentage arrays for all three
# Landsat collections
L5_dates = np.array(get_landsat_dates(L5_masked))
L5_clouds = get_landsat_clouds(L5_masked)
L7_dates = np.array(get_landsat_dates(L7_masked))
L7_clouds = get_landsat_clouds(L7_masked)
L8_dates = np.array(get_landsat_dates(L8_masked))
L8_clouds = get_landsat_clouds(L8_masked)
%cd '/content/drive/MyDrive/Research/landsat_scenes/dates_clouds'
np.savetxt("L5_dates.csv", L5_dates, delimiter=",", fmt='%s')
np.savetxt("L7_dates.csv", L7_dates, delimiter=",", fmt='%s')
np.savetxt("L8_dates.csv", L8_dates, delimiter=",", fmt='%s')
np.savetxt("L5_clouds.csv", np.array(L5_clouds), delimiter=",")
np.savetxt("L7_clouds.csv", np.array(L7_clouds), delimiter=",")
np.savetxt("L8_clouds.csv", np.array(L8_clouds), delimiter=",")

/content/drive/MyDrive/Research/landsat_scenes/dates_clouds


In [ ]:
# loads the date and cloud cover percentage arrays for all three Lansat
# collections
def readCSV(fname):
  file = open(fpath + fname + ".csv")
  csvreader = csv.reader(file)
  array = []
  for row in csvreader:
    array.append(row)
  return np.array(array)

fpath = '/content/drive/MyDrive/Research/landsat_scenes/dates_clouds/'
L5_dates = np.squeeze(readCSV("L5_dates"), 1)
L7_dates = np.squeeze(readCSV("L7_dates"), 1)
L8_dates = np.squeeze(readCSV("L8_dates"), 1)
L5_clouds = np.squeeze(readCSV("L5_clouds"), 1).astype(float)
L7_clouds = np.squeeze(readCSV("L7_clouds"), 1).astype(float)
L8_clouds = np.squeeze(readCSV("L8_clouds"), 1).astype(float)

NotADirectoryError: ignored

In [ ]:
# concatenates all of the date and cloud cover percentage arrays and saves
dates = np.concatenate((L5_dates, L7_dates, L8_dates))
clouds = np.concatenate((L5_clouds, L7_clouds, L8_clouds))
indices = np.argsort(dates)
clouds = clouds[indices]
dates = dates[indices]
%cd '/content/drive/MyDrive/Research/landsat_scenes/dates_clouds'
np.savetxt("dates.csv", dates, delimiter=",", fmt='%s')
np.savetxt("clouds.csv", clouds, delimiter=",")

/content/drive/MyDrive/Research/landsat_scenes/dates_clouds


# Downloading .csv Data

In [ ]:
# area of interest
aoi = ee.Geometry.Polygon([[[-71.45, 41.75],[-71.45, 41.25],[-71.15, 41.25],[-71.15, 41.75]]], None, False)

In [ ]:
# saves the .tif files for Landsat 5 (right now these will only save to the
# homepage of your google drive and must be moved manually)
for i in range(len(L5_dates)):
  image = L5_masked.filterDate(L5_dates[i], '2023-01-01').first().select("ST_B6")
  task_config = {
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': str(L5_dates[i]) + "_L5",
    'folder': "Landsat_5/",
    'image': image
  }
  task = ee.batch.Export.image.toDrive(**task_config)
  task.start()

In [ ]:
# saves the .tif files for Landsat 7 (right now these will only save to the
# homepage of your google drive and must be moved manually)
for i in range(len(L7_dates)):
  image = L7_masked.filterDate(L7_dates[i], '2023-01-01').first().select("ST_B6")
  task_config = {
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': str(L7_dates[i]) + "_L7",
    'folder': "Landsat_7/",
    'image': image
  }
  task = ee.batch.Export.image.toDrive(**task_config)
  task.start()

In [ ]:
# saves the .tif files for Landsat 8 (right now these will only save to the
# homepage of your google drive and must be moved manually)
for i in range(len(L8_dates)):
  image = L8_masked.filterDate(L8_dates[i], '2023-01-01').first().select("ST_B10")
  task_config = {
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': str(L8_dates[i]) + "_L8",
    'folder': "Landsat_8/",
    'image': image
  }
  task = ee.batch.Export.image.toDrive(**task_config)
  task.start()

In [ ]:
# applies the bias corrections found over the summer to each satellite
# collection and transforms the .tif files into csv files
L5_bias = -0.45039759427790915
L7_bias = -1.0943420408594975
L8_bias = 0.1777929159547842

%cd '/content/drive/MyDrive/Research/landsat_scenes/csv_scenes/landsat_5'
for i in range(len(L5_dates)):
  print("Lansat 5 Scene " + str(i+1) + " of " + str(len(L5_dates)))
  image_path = L5_dates[i] + "_L5.tif"
  file_path = '/content/drive/MyDrive/Research/landsat_scenes/tif_scenes/landsat_5/' + image_path
  raster = rasterio.open(file_path)
  thermal_array = np.array(raster.read(1))-273.15-L5_bias
  np.savetxt(L5_dates[i] + ".csv", thermal_array[20:1150,50:850], delimiter=",")
%cd '/content/drive/MyDrive/Research/landsat_scenes/csv_scenes/landsat_7'
for i in range(len(L7_dates)):
  print("Lansat 7 Scene " + str(i+1) + " of " + str(len(L7_dates)))
  image_path = L7_dates[i] + "_L7.tif"
  file_path = '/content/drive/MyDrive/Research/landsat_scenes/tif_scenes/landsat_7/' + image_path
  raster = rasterio.open(file_path)
  thermal_array = np.array(raster.read(1))-273.15-L7_bias
  thermal_array[thermal_array < -100] = np.nan
  np.savetxt(L7_dates[i] + ".csv", thermal_array[20:1150,50:850], delimiter=",")
%cd '/content/drive/MyDrive/Research/landsat_scenes/csv_scenes/landsat_8'
for i in range(len(L8_dates)):
  print("Lansat 8 Scene " + str(i+1) + " of " + str(len(L8_dates)))
  image_path = L8_dates[i] + "_L8.tif"
  file_path = '/content/drive/MyDrive/Research/landsat_scenes/tif_scenes/landsat_8/' + image_path
  raster = rasterio.open(file_path)
  thermal_array = np.array(raster.read(1))-273.15-L8_bias
  np.savetxt(L8_dates[i] + ".csv", thermal_array[20:1150,50:850], delimiter=",")

In [ ]:
# saves a Landsat 7 scene with no scanlines, used to find water pixels for dineof
L7_bias = -1.0943420408594975
i = 6
%cd '/content/drive/MyDrive/Research/landsat_scenes/csv_scenes'
image_path = L7_dates[i] + "_L7.tif"
file_path = '/content/drive/MyDrive/Research/landsat_scenes/tif_scenes/landsat_7/' + image_path
raster = rasterio.open(file_path)
thermal_array = np.array(raster.read(1))-273.15-L7_bias
thermal_array[thermal_array < -100] = np.nanmean(thermal_array)
np.savetxt(L7_dates[i] + "_full" ".csv", thermal_array[20:1150,50:850], delimiter=",")

/content/drive/MyDrive/Research/landsat_scenes/csv_scenes


# Lat / Lon Data

In [ ]:
# loads the date arrays for all three Landsat collections
def readCSV(fpath, fname):
  file = open(fpath + fname + ".csv")
  csvreader = csv.reader(file)
  array = []
  for row in csvreader:
    array.append(row)
  return np.array(array)

fpath1 = '/content/drive/MyDrive/Research/dineof_scenes/landsat_5/'
fpath2 = '/content/drive/MyDrive/Research/dineof_scenes/landsat_7/'
fpath3 = '/content/drive/MyDrive/Research/dineof_scenes/landsat_8/'

L5_dates = np.squeeze(readCSV(fpath1, "landsat_5_filtered_dates"), 1)
L7_dates = np.squeeze(readCSV(fpath2, "landsat_7_filtered_dates"), 1)
L8_dates = np.squeeze(readCSV(fpath3, "landsat_8_filtered_dates"), 1)

In [36]:
# downloads a lat and lon .tif image for Landsat 5
L5Masked = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')\
  .filter(ee.Filter.eq('WRS_PATH', 12))\
  .filter(ee.Filter.eq('WRS_ROW', 31))\
  .filterDate('1984-01-01', '2023-01-01')

aoi = ee.Geometry.Polygon([[[-71.45, 41.75],[-71.45, 41.25],[-71.15, 41.25],[-71.15, 41.75]]], None, False)
latlon = ee.Image.pixelLonLat()
image = L5Masked.filterDate(L5_dates[0], '2023-01-01').first()

lat = image.addBands(latlon).setDefaultProjection(image.projection()).select("latitude")
task_config_lat = {
  'region': aoi,
  'fileFormat': 'GeoTIFF',
  'fileNamePrefix': 'lat',
  'folder': 'latlon/',
  'image': lat
}
task_lat = ee.batch.Export.image.toDrive(**task_config_lat)
task_lat.start()

lon = image.addBands(latlon).setDefaultProjection(image.projection()).select("longitude")
task_config_lon = {
  'region': aoi,
  'fileFormat': 'GeoTIFF',
  'fileNamePrefix': 'lon',
  'folder': 'latlon/',
  'image': lon
}
task_lon = ee.batch.Export.image.toDrive(**task_config_lon)
task_lon.start()

In [42]:
# converts the .tif lat and lon images to csv and saves to drive
raster_lat = rasterio.open('/content/drive/MyDrive/Research/lat_lon/landsat_5/lat.tif')
raster_lon = rasterio.open('/content/drive/MyDrive/Research/lat_lon/landsat_5/lon.tif')
lat_array = np.array(raster_lat.read(1))
lon_array = np.array(raster_lon.read(1))
%cd '/content/drive/MyDrive/Research/lat_lon/landsat_5'
np.savetxt("lat_5.csv", lat_array[20:1150,50:850], delimiter=",")
np.savetxt("lon_5.csv", lon_array[20:1150,50:850], delimiter=",")

/content/drive/MyDrive/Research/lat_lon/landsat_5


In [43]:
# downloads a lat and lon .tif image for Landsat 7
L7Masked = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
  .filter(ee.Filter.eq('WRS_PATH', 12))\
  .filter(ee.Filter.eq('WRS_ROW', 31))\
  .filterDate('1984-01-01', '2023-01-01')

aoi = ee.Geometry.Polygon([[[-71.45, 41.75],[-71.45, 41.25],[-71.15, 41.25],[-71.15, 41.75]]], None, False)
latlon = ee.Image.pixelLonLat()
image = L7Masked.filterDate(L7_dates[0], '2023-01-01').first()

lat = image.addBands(latlon).setDefaultProjection(image.projection()).select("latitude")
task_config_lat = {
  'region': aoi,
  'fileFormat': 'GeoTIFF',
  'fileNamePrefix': 'lat',
  'folder': 'latlon/',
  'image': lat
}
task_lat = ee.batch.Export.image.toDrive(**task_config_lat)
task_lat.start()

lon = image.addBands(latlon).setDefaultProjection(image.projection()).select("longitude")
task_config_lon = {
  'region': aoi,
  'fileFormat': 'GeoTIFF',
  'fileNamePrefix': 'lon',
  'folder': 'latlon/',
  'image': lon
}
task_lon = ee.batch.Export.image.toDrive(**task_config_lon)
task_lon.start()

In [49]:
# converts the .tif lat and lon images to csv and saves to drive
raster_lat = rasterio.open('/content/drive/MyDrive/Research/lat_lon/landsat_7/lat.tif')
raster_lon = rasterio.open('/content/drive/MyDrive/Research/lat_lon/landsat_7/lon.tif')
lat_array = np.array(raster_lat.read(1))
lon_array = np.array(raster_lon.read(1))
%cd '/content/drive/MyDrive/Research/lat_lon/landsat_7'
np.savetxt("lat_7.csv", lat_array[20:1150,50:850], delimiter=",")
np.savetxt("lon_7.csv", lon_array[20:1150,50:850], delimiter=",")

/content/drive/MyDrive/Research/lat_lon/landsat_7


In [50]:
# downloads a lat and lon .tif image for Landsat 8
L8Masked = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
  .filter(ee.Filter.eq('WRS_PATH', 12))\
  .filter(ee.Filter.eq('WRS_ROW', 31))\
  .filterDate('1984-01-01', '2023-01-01')

aoi = ee.Geometry.Polygon([[[-71.45, 41.75],[-71.45, 41.25],[-71.15, 41.25],[-71.15, 41.75]]], None, False)
latlon = ee.Image.pixelLonLat()
image = L8Masked.filterDate(L8_dates[0], '2023-01-01').first()

lat = image.addBands(latlon).setDefaultProjection(image.projection()).select("latitude")
task_config_lat = {
  'region': aoi,
  'fileFormat': 'GeoTIFF',
  'fileNamePrefix': 'lat',
  'folder': 'latlon/',
  'image': lat
}
task_lat = ee.batch.Export.image.toDrive(**task_config_lat)
task_lat.start()

lon = image.addBands(latlon).setDefaultProjection(image.projection()).select("longitude")
task_config_lon = {
  'region': aoi,
  'fileFormat': 'GeoTIFF',
  'fileNamePrefix': 'lon',
  'folder': 'latlon/',
  'image': lon
}
task_lon = ee.batch.Export.image.toDrive(**task_config_lon)
task_lon.start()

In [57]:
# converts the .tif lat and lon images to csv and saves to drive
raster_lat = rasterio.open('/content/drive/MyDrive/Research/lat_lon/landsat_8/lat.tif')
raster_lon = rasterio.open('/content/drive/MyDrive/Research/lat_lon/landsat_8/lon.tif')
lat_array = np.array(raster_lat.read(1))
lon_array = np.array(raster_lon.read(1))
%cd '/content/drive/MyDrive/Research/lat_lon/landsat_8'
np.savetxt("lat_8.csv", lat_array[20:1150,50:850], delimiter=",")
np.savetxt("lon_8.csv", lon_array[20:1150,50:850], delimiter=",")

/content/drive/MyDrive/Research/lat_lon/landsat_8
